<a href="https://colab.research.google.com/github/Yason-9/2023_zfish_research/blob/main/mouse_dlc_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install https://github.com/DeepLabCut/DeepLabCut/archive/master.zip


     / 75.5 MB 32.8 MB/s 0:00:04
  Preparing metadata (setup.py) ... done


In [7]:
import requests
from io import BytesIO
from zipfile import ZipFile

url_record = 'https://zenodo.org/api/records/7883589'
response = requests.get(url_record)
if response.status_code == 200:
    file = response.json()['files'][0]
    title = file['key']
    print(f"Downloading {title}...")
    with requests.get(file['links']['self'], stream=True) as r:
        with ZipFile(BytesIO(r.content)) as zf:
            zf.extractall(path='/content')
else:
    raise ValueError(f'The URL {url_record} could not be reached.')

In [9]:
import deeplabcut as dlc
import os

project_path = "/content/demo-me-2021-07-14"
config_path = os.path.join(project_path, "config.yaml")
video = os.path.join(project_path, "videos", "videocompressed1.mp4")

dlc.analyze_videos(config_path,[video], shuffle=0, videotype="mp4",auto_track=False )


Using snapshot-20000 for model /content/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0


/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '


Activating extracting of PAFs
Starting to analyze %  /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Loading  /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Duration of video [s]:  77.67 , recorded with  30.0 fps!
Overall # of frames:  2330  found with (before cropping) frame dimensions:  640 480
Starting to extract posture from the video(s) with batchsize: 8


100%|██████████| 2330/2330 [02:28<00:00, 15.64it/s]


Video Analyzed. Saving results in /content/demo-me-2021-07-14/videos...
The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_dlcrnetms5_demoJul14shuffle0_20000'

In [11]:
TRACK_METHOD = "ellipse"
dlc.convert_detections2tracklets(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    track_method=TRACK_METHOD,
    ignore_bodyparts=["tail1", "tail2", "tailend"]
)

#deconstruct video into spatio-temporal grouping
#track body part assemblies frame-by-frame

Using snapshot-20000 for model /content/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0
Processing...  /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Analyzing /content/demo-me-2021-07-14/videos/videocompressed1DLC_dlcrnetms5_demoJul14shuffle0_20000.h5


100%|██████████| 2330/2330 [00:13<00:00, 178.64it/s]
2330it [00:13, 175.05it/s]


The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.


In [12]:
dlc.stitch_tracklets(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    track_method=TRACK_METHOD,
    n_tracks=3
)

/usr/local/lib/python3.10/dist-packages/deeplabcut/refine_training_dataset/stitch.py:138: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  return mode(self.data[..., 3], axis=None, nan_policy="omit")[0][0]


Processing...  /content/demo-me-2021-07-14/videos/videocompressed1.mp4


100%|██████████| 3/3 [00:00<00:00, 25015.73it/s]


In [13]:
dlc.filterpredictions(
    config_path,
    [video],
    shuffle=0,
    videotype='mp4',
    track_method = TRACK_METHOD
)

dlc.create_labeled_video(
    config_path,
    [video],
    videotype='mp4',
    shuffle=0,
    color_by="individual",
    keypoints_only=False,
    draw_skeleton=True,
    filtered=True,
    track_method=TRACK_METHOD,
)

Filtering with median model /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Saving filtered csv poses!
Starting to process video: /content/demo-me-2021-07-14/videos/videocompressed1.mp4
Loading /content/demo-me-2021-07-14/videos/videocompressed1.mp4 and data.
Duration of video [s]: 77.67, recorded with 30.0 fps!
Overall # of frames: 2330 with cropped frame dimensions: 640 480
Generating frames and creating video.


100%|██████████| 2330/2330 [01:05<00:00, 35.47it/s]


[True]